In [2]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import json

In [2]:
# url_list = [
#     "https://www.zillow.com/homedetails/516-Jewett-Ave-Staten-Island-NY-10302/32282043_zpid/",
#     "https://www.zillow.com/homedetails/131-Rockland-Ave-Staten-Island-NY-10306/32295690_zpid/",
#     "https://www.zillow.com/homedetails/254-Edgegrove-Ave-Staten-Island-NY-10312/32364971_zpid/",
#     "https://www.zillow.com/homedetails/104-Burton-Ave-Staten-Island-NY-10309/32368184_zpid/",
#     "https://www.zillow.com/homedetails/51-Lexington-Ave-Staten-Island-NY-10302/32296475_zpid/"
# ]


# print(url_list)

In [3]:
url_list = []

with open('house_urls_list.txt', 'r') as f: #open the file 
    url_list = f.readlines() # combine lines of file in a list

new_url_list =[]

for url in url_list:
    url = url.replace('\n', '')
    
    new_url_list.append(url)


new_url_list

['https://www.zillow.com/homedetails/580-Carroll-St-1A-Brooklyn-NY-11215/145588170_zpid/',
 'https://www.zillow.com/homedetails/70-25-Yellowstone-Blvd-20Y-Forest-Hills-NY-11375/2071118635_zpid/',
 'https://www.zillow.com/homedetails/19018-114th-Dr-Saint-Albans-NY-11412/32143487_zpid/',
 'https://www.zillow.com/homedetails/10617-154th-St-Jamaica-NY-11433/32137985_zpid/',
 'https://www.zillow.com/homedetails/40-Danny-Ct-Staten-Island-NY-10314/62723452_zpid/',
 'https://www.zillow.com/homedetails/145-Leverett-Ave-Staten-Island-NY-10308/32342198_zpid/',
 'https://www.zillow.com/homedetails/446-Westervelt-Ave-Staten-Island-NY-10301/112116219_zpid/',
 'https://www.zillow.com/homedetails/358-74th-St-Brooklyn-NY-11209/30694934_zpid/',
 'https://www.zillow.com/homedetails/43-Paris-Ct-Staten-Island-NY-10310/316675090_zpid/',
 'https://www.zillow.com/homedetails/95-15-Pitkin-Ave-Ozone-Park-NY-11417/2071120524_zpid/',
 'https://www.zillow.com/homedetails/12-Franklin-Ln-APT-G-Staten-Island-NY-10306

In [4]:
len(new_url_list)

1565

In [37]:
def get_html(url):
    """This function takes a Zillow url, and returns HTML code for that URL."""

    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }

    # make a request to Zillow
    r = requests.get(url, headers=req_headers)
    content = r.content
    
    return content

In [23]:
def convert_to_soup(html):
    """Takes HTML object and converts to soup using BeautifulSoup"""
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

In [24]:
def price(soup):
    """Takes a Zillow soup object, and tries to return the price."""
    try:
        price = soup.find(class_='ds-summary-row').find(class_="Text-c11n-8-33-0__aiai24-0 sc-pkUbs dbPIZT").text
        price=price.replace('$', '')
        price=price.replace(',', '')
        price=int(price)

        return price
    
    except:
        return "N/A"


def bedrooms(soup):
    """Uses soup object to get bedrooms"""
    try:
        bed=soup.find(class_='ds-summary-row').find(class_="sc-psQdR iYtUny").text
        bed=bed.split(' ')[0]
        bed=int(bed)

        return bed
    
    except:
        return "N/A"


def bathrooms(soup):
    try:
        bath=soup.find(class_='ds-summary-row').find_all(class_="sc-psQdR iYtUny")[1].text
        bath=bath.split(' ')[0]
        bath=int(bath)

        return bath
    
    except:
        return "N/A"


def sqft(soup):
    try:
        sqft=soup.find(class_='ds-summary-row').find_all(class_="sc-psQdR iYtUny")[2].text
        sqft=sqft.split(' ')[0]
        sqft=sqft.replace(',', '')
        sqft=int(sqft)

        return(sqft)
    
    except:
        return "N/A"


def address(soup):
    try:
    
        address=soup.find_all(class_='Text-c11n-8-33-0__aiai24-0 eAgJuz ds-price-change-address-row')
        address=address[1].text

        return(address)
    
    except:
        return "N/A"


def house_type(soup):
    try:
        house_type=soup.find_all(class_='ds-expandable-card-section-default-padding')[1].select("span.Text-c11n-8-33-0__aiai24-0")[1].text

        return(house_type)
    
    except:
        return "N/A"



def year_built(soup):
    try:
        year_built=soup.find_all(class_='ds-expandable-card-section-default-padding')[1].select("span.Text-c11n-8-33-0__aiai24-0")[3].text

        return(year_built)
    
    except:
        return "N/A"



def heating(soup):
    try:
        heating=soup.find_all(class_='ds-expandable-card-section-default-padding')[1].select("span.Text-c11n-8-33-0__aiai24-0")[5].text

        return(heating)
    except:
        return "N/A"


def cooling(soup):
    try:
        cooling=soup.find_all(class_='ds-expandable-card-section-default-padding')[1].select("span.Text-c11n-8-33-0__aiai24-0")[7].text

        return cooling
    except:
        return "N/A"

def parking(soup):
    try:
        parking=soup.find_all(class_='ds-expandable-card-section-default-padding')[1].select("span.Text-c11n-8-33-0__aiai24-0")[9].text

        return parking
    except:
        return "N/A"

def lot_size(soup):
    try:
        # there is a script tag in Zillow home pages that includes property details
        # this finds the script tag with the hdpApolloPreloadedData id
        # loads the content into a dictionary, which we call api_features_dict
        script = soup.select("script#hdpApolloPreloadedData")[0]
        script = script.contents[0]
        script_json = json.loads(script)
        features_string = script_json['apiCache']
        api_features_dict = json.loads(features_string)

        # api_features_dict has two keys, but they include the zillow property id
        # in the key, which we would need to search for manually
        # Instead, we'll just get the list of keys for api_features_dict
        # and search for the first key, which contains property details
        api_key = list(api_features_dict.keys())[0]

        # Create a variable to contain all property details
        property_details = api_features_dict[api_key]['property']

        # Get lotsize from property details
        lot_size = property_details['lotSize']

        return lot_size
    
    except:
        return "N/A"


def zestimate(soup):
    try:
        zestimate=soup.find_all(class_='Text-c11n-8-33-0__aiai24-0 LoSpP')[1].text
        zestimate=zestimate.replace('$', '')
        zestimate=zestimate.replace(',', '')
        zestimate=int(zestimate)
        return zestimate
    except: 
        return "N/A"


def zip_code(address):
    try:
        zip_code = address[-5:]

        return zip_code
    except:
        return "N/A"


def num_photos(soup):
    try:
        ### re-pasting the code from lot_size because this will be a new function
        # there is a script tag in Zillow home pages that includes property details
        # this finds the script tag with the hdpApolloPreloadedData id
        # loads the content into a dictionary, which we call api_features_dict
        script = soup.select("script#hdpApolloPreloadedData")[0]
        script = script.contents[0]
        script_json = json.loads(script)
        features_string = script_json['apiCache']
        api_features_dict = json.loads(features_string)


        # api_features_dict has two keys, but they include the zillow property id
        # in the key, which we would need to search for manually
        # Instead, we'll just get the list of keys for api_features_dict
        # and search for the second key, which contains photos
        api_key_2 = list(api_features_dict.keys())[1]

        # Create a variable to contain all photos
        photos = api_features_dict[api_key_2]['property']['responsivePhotos']

        num_photos = len(photos)    

        return num_photos

    except:
        return "N/A"


In [25]:
def get_zillow_data(url):
    # get html for the url and convert to soup
    html = get_html(url)
    soup = convert_to_soup(html)
    
    the_address = address(soup)
    
    # parse soup for features
    dict_of_features={
        "price": price(soup),
        "bed"  : bedrooms(soup),
        "bath" : bathrooms(soup),
        "sqft" : sqft(soup),
        "address" : the_address,
        "zip_code" : zip_code(the_address),
        "house_type" : house_type(soup),
        "year_built" : year_built(soup),
        "heating" : heating(soup),
        "cooling" : cooling(soup),
        "parking" : parking(soup),
        "lot_size" : lot_size(soup),
        "zestimate" : zestimate(soup),
        "num_photos" : num_photos(soup),
        "url": url
       
    }
    
    
    return dict_of_features
    


Separating rows of urls into grouped dictionaries, appending dictionaries into a dataframe: 

In [46]:
first_list_of_dictionaries = []

for url in new_url_list[:400]:
    first_list_of_dictionaries.append(get_zillow_data(url))

In [50]:
second_list_of_dictionaries = []

for url in new_url_list[400:800]:
    second_list_of_dictionaries.append(get_zillow_data(url))

In [51]:
third_list_of_dictionaries = []

for url in new_url_list[800:1200]:
    third_list_of_dictionaries.append(get_zillow_data(url))

In [52]:
fourth_list_of_dictionaries = []

for url in new_url_list[1200:]:
    fourth_list_of_dictionaries.append(get_zillow_data(url))

In [55]:
final_list_of_dictionaries = []
final_list_of_dictionaries.extend(first_list_of_dictionaries)
final_list_of_dictionaries.extend(second_list_of_dictionaries)
final_list_of_dictionaries.extend(third_list_of_dictionaries)
final_list_of_dictionaries.extend(fourth_list_of_dictionaries)

len(final_list_of_dictionaries)

1565

In [61]:
# save data in text file
with open('zillow_house_features.txt', 'w') as f:
    for item in final_list_of_dictionaries:
        f.write("%s\n" % item)

In [57]:
house_df=pd.DataFrame(final_list_of_dictionaries)
house_df.tail(2)

,price,bed,bath,sqft,address,zip_code,house_type,year_built,heating,cooling,parking,lot_size,zestimate,num_photos,url
1563,2500000,5,3,2280,"220 Adelphi St, Brooklyn, NY 11205",11205,MultiFamily,1901,No Data,Central,0 spaces,2000,2377377,12,https://www.zillow.com/homedetails/220-Adelphi...
1564,599900,3,3,1488,"78 Garretson Ln, Staten Island, NY 10304",10304,Single Family - Attached,1999,"Forced Air, Natural Gas",Central Air,"Built-in, Garage, Off Street, On Street",2040,585617,46,https://www.zillow.com/homedetails/78-Garretso...


In [62]:
house_df.to_csv('zillow_house_features.csv', index=False)

In [75]:
tmp_df = house_df.copy()

tmp_df = tmp_df.replace("N/A", np.nan)

In [76]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       1562 non-null   float64
 1   bed         1505 non-null   float64
 2   bath        1548 non-null   float64
 3   sqft        1371 non-null   float64
 4   address     1563 non-null   object 
 5   zip_code    1563 non-null   object 
 6   house_type  1553 non-null   object 
 7   year_built  1553 non-null   object 
 8   heating     1553 non-null   object 
 9   cooling     1553 non-null   object 
 10  parking     1553 non-null   object 
 11  lot_size    1241 non-null   float64
 12  zestimate   1165 non-null   float64
 13  num_photos  1564 non-null   float64
 14  url         1565 non-null   object 
dtypes: float64(7), object(8)
memory usage: 183.5+ KB


In [77]:
tmp_df.dropna(subset=["lot_size", "sqft"], inplace=True)

In [78]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1155 entries, 0 to 1564
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       1155 non-null   float64
 1   bed         1136 non-null   float64
 2   bath        1150 non-null   float64
 3   sqft        1155 non-null   float64
 4   address     1155 non-null   object 
 5   zip_code    1155 non-null   object 
 6   house_type  1155 non-null   object 
 7   year_built  1155 non-null   object 
 8   heating     1155 non-null   object 
 9   cooling     1155 non-null   object 
 10  parking     1155 non-null   object 
 11  lot_size    1155 non-null   float64
 12  zestimate   931 non-null    float64
 13  num_photos  1155 non-null   float64
 14  url         1155 non-null   object 
dtypes: float64(7), object(8)
memory usage: 144.4+ KB


In [48]:
combined_house_df = pd.concat()

,price,bed,bath,sqft,address,zip_code,house_type,year_built,heating,cooling,parking,lot_size,zestimate,num_photos,url
375,899000,3,2,2544,"16 Rockville Ave, Staten Island, NY 10314",10314,Two Family - Detached,1973,"Forced Air, Natural Gas",None,"No Garage, Off Street",4000,873248,40,https://www.zillow.com/homedetails/16-Rockvill...
376,885000,5,3,2904,"172 Sommer Ave, Staten Island, NY 10314",10314,Two Family - Detached,1975,"Forced Air, Natural Gas",Central Air,"Attached, Detached, Garage, Off Street, On Street",5100,864369,35,https://www.zillow.com/homedetails/172-Sommer-...
377,445888,2,2,1248,"Price cut: $4K (5/5)91 Mcdivitt Ave, Staten Is...",10314,Single Family - Attached,1992,"Forced Air, Natural Gas",Central Air,"No Garage, Assigned",2477,437559,15,https://www.zillow.com/homedetails/91-Mcdivitt...
378,350000,N/A,1,N/A,"206 E 7th St APT 3, New York, NY 10009",10009,Cooperative,1900,No Data,No Data,0 spaces,2272,338838,8,https://www.zillow.com/homedetails/206-E-7th-S...
379,561998,3,2,1572,"9 Pamela Ln, Staten Island, NY 10304",10304,Single Family - Attached,1982,"Forced Air, Natural Gas",Central Air,"No Garage, Off Street",1600,544301,20,https://www.zillow.com/homedetails/9-Pamela-Ln...
380,798000,4,3,2600,"2684 E 63rd St, Brooklyn, NY 11234",11234,SingleFamily,1960,No Data,No Data,0 spaces,2600,N/A,20,https://www.zillow.com/homedetails/2684-E-63rd...
381,465000,3,2,1192,"102 Church Ave, Staten Island, NY 10314",10314,Single Family Residence,1989,"Natural Gas, Other",Central Air,"Private Drive, 2 Spaces, Built-In Garage",750,454312,1,https://www.zillow.com/homedetails/102-Church-...
382,195000,N/A,1,500,"112-20 72nd Dr #D18, Forest Hills, NY 11375",11375,Stock Cooperative,1950,Oil,No Data,Waitlist,N/A,199778,11,https://www.zillow.com/homedetails/112-20-72nd...
383,249000,2,1,N/A,"40 E 43rd St APT 3B, Brooklyn, NY 11203",11203,Stock Cooperative,1957,"Natural Gas, Steam",No Data,0 spaces,N/A,N/A,14,https://www.zillow.com/homedetails/40-E-43rd-S...
384,649000,5,3,1640,"553 Saratoga Ave, Brooklyn, NY 11212",11212,SingleFamily,1987,No Data,Central,0 spaces,1999,649104,21,https://www.zillow.com/homedetails/553-Saratog...
